<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-18 14:46:05
-------------------
qualified stocks: 94
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  1.01 L
Current:  1.36 C
-------------------
Today PnL: 2.22 L (1.65%)
Current PnL: -18.56 L (-12.62%)
CY Booked + Current PnL: -10.29 L (-7.0%)
-------------------
Total profit:  3.44 L
Total loss:  -22.00 L
-------------------
Total Booked + Current PnL: 16.69 L (13.9%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.07%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.18 L (57.35%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,6.57,114360.0,10048.0,6599.0,2.80,9.63,5.77,15.96,9.0,1.52,0.85,28.16,X5K,ATH,METALS
69,SBIN,760.30,863.00,-11.34,M-LC,3.87,219857.0,18378.0,8838.0,0.37,9.12,4.02,13.51,60.0,2.08,1.62,22.87,XY25,NTT,BANKS
52,LALPATHLAB,2838.53,3545.00,-22.49,H-SC,10.89,231438.0,38418.0,9628.0,1.66,19.90,4.16,24.89,137.0,3.99,1.71,47.90,X40N,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.24,172282.0,16105.0,9992.0,0.12,10.31,5.80,16.71,22.0,1.61,1.27,29.38,X40N,BTT,HEALTHCARE
47,ITC,411.90,452.00,-38.34,H-LC,2.79,148567.0,-1365.0,15956.0,-0.80,-0.91,10.74,9.74,4.0,-0.09,1.10,5.27,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RAJOOENG,102.50,143.33,-32.80,H-SC,6.05,101980.0,-520.0,41353.0,-0.57,-0.51,40.55,39.83,114.0,-0.01,0.75,0.97,AR,ATH,MISC
59,PGHH,13388.00,18062.58,-30.55,H-MC,6.25,201105.0,285.0,69824.0,1.05,0.14,34.72,34.92,80.0,0.00,1.49,4.82,X40,ATH,FMCG
57,NATIONALUM,189.63,247.44,-50.29,H-MC,1.99,99942.0,-562.0,31202.0,0.53,-0.56,31.22,30.49,79.0,-0.02,0.74,31.66,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.34,H-LC,2.79,148567.0,-1365.0,15956.0,-0.80,-0.91,10.74,9.74,4.0,-0.09,1.10,5.27,X40,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.44,H-LC,2.82,215090.0,10590.0,30822.0,0.38,5.18,14.33,20.25,10.0,0.34,1.59,14.36,X40N,BTT,PHARMA
58,NESTLEIND,1134.30,1377.0,-12.68,H-LC,3.34,268819.0,3393.0,53387.0,5.45,1.28,19.86,21.40,11.0,0.06,1.99,8.46,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-9.79,H-LC,3.73,175923.0,-25463.0,113083.0,2.31,-12.64,64.28,43.51,27.0,-0.23,1.30,22.46,X40,BTT,PAINTS
54,LTIM,5564.16,7230.2,-8.38,H-LC,3.76,182016.0,-18294.0,78267.0,-1.02,-9.13,43.00,29.94,16.0,-0.23,1.35,25.75,X200,ATH,IT


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,RELAXO,902.64,1176.00,-34.84,H-SC,10.66,60448.0,-55090.0,90080.0,8.17,-47.68,149.02,30.28,136.0,-0.61,0.45,18.09,X40N,NTT,FOOTWEAR
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,6.57,191860.0,27190.0,40770.0,7.47,16.51,21.25,41.28,89.0,0.67,1.42,25.81,X40N,ATH,AC
13,BATAINDIA,1550.24,2096.00,-37.72,M-SC,7.46,93168.0,-35502.0,80805.0,6.94,-27.59,86.73,35.20,219.0,-0.44,0.69,6.94,X40,NTT,FOOTWEAR
91,VOLTAS,1278.28,1918.49,-2.76,H-MC,7.64,204030.0,12288.0,83734.0,6.70,6.41,41.04,50.08,99.0,0.15,1.51,13.64,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,8.58,174510.0,23597.0,33715.0,5.90,15.64,19.32,37.97,93.0,0.70,1.29,44.90,AR,ATH,AUTO


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,MEDANTA,1087.93,1486.00,9.57,H-MC,13.86,156322.0,30122.0,16054.0,-1.93,23.87,10.27,36.59,108.0,1.88,1.16,39.37,XY24,NTT,HEALTHCARE
74,SONACOMS,578.05,1006.00,-37.35,M-SC,13.62,77919.0,-23240.0,98131.0,-1.46,-22.97,125.94,74.03,202.0,-0.24,0.58,9.87,AR,BTT,AUTO
30,GREENPANEL,375.16,537.00,194.57,M-SC,5.42,131565.0,-51513.0,130486.0,-1.30,-28.14,99.18,43.14,230.0,-0.39,0.97,23.75,XY24,NTT,MISC
72,SIEMENS,4406.90,7969.85,-2.61,H-LC,35.01,78280.0,-31892.0,120966.0,-1.19,-28.95,154.53,80.85,15.0,-0.26,0.58,15.29,AR,ATH,ELECTRICAL
54,LTIM,5564.16,7230.20,-8.38,H-LC,3.76,182016.0,-18294.0,78267.0,-1.02,-9.13,43.00,29.94,16.0,-0.23,1.35,25.75,X200,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,TTKPRESTIG,769.29,770.0,96.16,M-SC,10.13,84482.0,-16295.0,16390.0,2.09,-16.17,19.40,0.09,245.0,-0.99,0.62,10.99,OX40N,NTT,DURABLES
20,CERA,7989.07,9475.0,-17.90,H-SC,3.14,117288.0,-26515.0,53260.0,1.84,-18.44,45.41,18.60,149.0,-0.50,0.87,29.43,OX40N,NTT,CERAMICS
85,UJJIVANSFB,52.77,60.0,84.36,H-SC,20.15,116181.0,-26298.0,45822.0,3.26,-18.46,39.44,13.70,163.0,-0.57,0.86,37.13,OX40N,NTT,BANKS
43,INDIGOPNTS,1407.73,1408.0,115.96,M-SC,2.55,141682.0,-32877.0,32913.0,3.20,-18.83,23.23,0.02,221.0,-1.00,1.05,22.86,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.07,H-SC,3.05,213201.0,-56466.0,92806.0,0.61,-20.94,43.53,13.47,138.0,-0.61,1.58,7.83,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TCS,3794.03,4998.00,-29.01,H-LC,16.11,234866.0,-61068.0,154988.0,-0.37,-20.64,65.99,31.73,1.0,-0.39,1.74,0.27,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.39,H-LC,11.27,257044.0,-4557.0,150191.0,-0.81,-1.74,58.43,55.67,3.0,-0.03,1.90,4.26,X40,BTT,IT
47,ITC,411.90,452.00,-38.34,H-LC,2.79,148567.0,-1365.0,15956.0,-0.80,-0.91,10.74,9.74,4.0,-0.09,1.10,5.27,X40,NTT,FMCG
90,VBL,492.64,671.64,-11.46,H-LC,10.05,268973.0,3933.0,92365.0,-0.38,1.48,34.34,36.33,5.0,0.04,1.99,14.98,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.40,H-LC,10.83,195741.0,-20778.0,113686.0,-0.22,-9.60,58.08,42.91,7.0,-0.18,1.45,2.51,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,MEDANTA,1087.93,1486.0,9.57,H-MC,13.86,156322.0,30122.0,16054.0,-1.93,23.87,10.27,36.59,108.0,1.88,1.16,39.37,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-10.29,H-LC,6.65,223930.0,39449.0,30925.0,4.86,21.38,13.81,38.14,38.0,1.28,1.66,34.26,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,6.57,191860.0,27190.0,40770.0,7.47,16.51,21.25,41.28,89.0,0.67,1.42,25.81,X40N,ATH,AC
56,MEDANTA,1087.93,1486.00,9.57,H-MC,13.86,156322.0,30122.0,16054.0,-1.93,23.87,10.27,36.59,108.0,1.88,1.16,39.37,XY24,NTT,HEALTHCARE
51,KPIGREEN,497.21,731.35,9.02,H-SC,11.63,130070.0,4773.0,54226.0,1.47,3.81,41.69,47.09,141.0,0.09,0.96,62.79,MH,ATH,POWER
91,VOLTAS,1278.28,1918.49,-2.76,H-MC,7.64,204030.0,12288.0,83734.0,6.70,6.41,41.04,50.08,99.0,0.15,1.51,13.64,XY25,ATH,AC
19,CAMS,3643.00,5211.76,-3.83,H-SC,4.19,109206.0,7202.0,36726.0,0.82,7.06,33.63,43.06,122.0,0.20,0.81,25.38,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,8.58,174510.0,23597.0,33715.0,5.90,15.64,19.32,37.97,93.0,0.70,1.29,44.90,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.09,H-SC,11.77,136358.0,13022.0,120745.0,1.06,10.56,88.55,108.45,119.0,0.11,1.01,35.27,AR,ATH,MISC
51,KPIGREEN,497.21,731.35,9.02,H-SC,11.63,130070.0,4773.0,54226.0,1.47,3.81,41.69,47.09,141.0,0.09,0.96,62.79,MH,ATH,POWER
92,WHIRLPOOL,1219.98,2270.00,-42.31,M-SC,4.52,97500.0,6002.0,72754.0,4.78,6.56,74.62,86.07,223.0,0.08,0.72,41.04,XR,NTT,DURABLES
93,WIPRO,243.46,420.00,-13.34,M-LC,7.62,152470.0,1525.0,107934.0,-0.36,1.01,70.79,72.51,53.0,0.01,1.13,7.09,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.48,74592.0,-38957.0,79008.0,2.34,-34.31,105.92,35.27,268.0,-0.49,0.55,82.57,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.16,H-SC,15.39,90154.0,-10809.0,105336.0,1.12,-10.71,116.84,93.63,148.0,-0.10,0.67,34.28,SR,ATH,CHEMICALS
55,MASFIN,326.60,399.50,-18.70,H-SC,13.22,92370.0,-5610.0,27480.0,2.46,-5.73,29.75,22.32,152.0,-0.20,0.68,33.97,XR,ATH,FINANCE
89,VALIANTORG,512.64,838.00,-325.83,H-SC,16.43,95966.0,-37320.0,121915.0,0.15,-28.00,127.04,63.47,139.0,-0.31,0.71,60.86,XR,NTT,CHEMICALS
9,BAJAJFINSV,1994.80,2758.00,-8.77,H-LC,7.22,98015.0,270.0,37128.0,3.91,0.28,37.88,38.26,35.0,0.01,0.72,29.02,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-5.85,H-MC,8.58,174510.0,23597.0,33715.0,5.90,15.64,19.32,37.97,93.0,0.70,1.29,44.90,AR,ATH,AUTO
52,LALPATHLAB,2838.53,3545.00,-22.49,H-SC,10.89,231438.0,38418.0,9628.0,1.66,19.90,4.16,24.89,137.0,3.99,1.71,47.90,X40N,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-10.29,H-LC,6.65,223930.0,39449.0,30925.0,4.86,21.38,13.81,38.14,38.0,1.28,1.66,34.26,X40N,NTT,FMCG
5,ASIANPAINT,2961.56,4250.00,-9.79,H-LC,3.73,175923.0,-25463.0,113083.0,2.31,-12.64,64.28,43.51,27.0,-0.23,1.30,22.46,X40,BTT,PAINTS
4,APOLLOHOSP,7098.95,8285.00,-13.75,H-LC,6.24,172282.0,16105.0,9992.0,0.12,10.31,5.80,16.71,22.0,1.61,1.27,29.38,X40N,BTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.37
1,25,40.92
2,50,70.51


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.50
LC    35.47
MC    26.05
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.09
X40N     15.40
X40      14.21
XY25     11.51
XR        8.86
AR        8.23
OX40N     7.35
X5K       2.22
MH        1.70
X200      1.35
SR        1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.33
H-SC    23.99
H-MC    23.16
M-SC    13.06
M-LC     7.08
M-MC     2.56
L-SC     1.45
L-LC     1.06
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.06,-1.83,33.01
IT,11.61,-18.91,78.30
FINANCE,8.05,-19.74,64.04
BANKS,7.44,-13.12,61.15
MISC,5.72,-18.69,86.72
PAINTS,5.55,-16.30,39.22
HEALTHCARE,5.18,6.30,17.36
ELECTRICAL,4.66,-11.52,59.81
AUTO,4.46,-12.05,60.80


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2833131.0,23
AR,1097758.0,10
XR,1094558.0,13
X40,814075.0,11
X40N,526163.0,12
XY25,491618.0,8
OX40N,463896.0,10
SR,194655.0,2
X5K,138323.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2772321.0,28
M-SC,1476457.0,17
H-MC,1245469.0,18
H-LC,1207187.0,19
M-LC,419643.0,5
M-MC,331429.0,2
L-SC,264725.0,3
L-MC,57577.0,1
L-LC,43064.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      936859.0      6
M-SC       XY24      865028.0      7
H-SC       AR        667919.0      5
H-LC       X40       527037.0      6
H-SC       XR        516824.0      6
H-MC       XY24      489009.0      5
M-MC       XY24      331429.0      2
H-LC       AR        234652.0      2
           X40N      208459.0      5
H-MC       X40       206233.0      4
           XY25      202827.0      2
H-SC       OX40N     202651.0      4
           X40N      199187.0      4
           SR        194655.0      2
L-SC       XR        174773.0      2
M-LC       XY24      171147.0      2
M-SC       AR        161472.0      2
M-LC       X5K       131724.0      1
M-SC       XY25      124375.0      1
           OX40N     122869.0      4
           XR        121908.0      2
H-MC       X40N      118517.0      3
           XR        115542.0      1
H-LC       XY25      112514.0      3
M-LC       XR        107934.0      1
L-SC       OX40N      89952.0      1
M-SC       X40        80805.0      1
H-LC       X200       78267.0      1
L-MC       XR         57577.0      1
H-SC       MH         54226.0      1
H-MC       OX40N      48424.0      1
L-LC       XY25       43064.0      1
H-LC       XY24       39659.0      1
H-MC       AR         33715.0      1
           MH         31202.0      1
M-LC       XY25        8838.0      1
H-LC       X5K         6599.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
